In [1]:
#import packages
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
#This code uses MNIST dataset for CNN.you need to change the data loading code for other datasets

#Loading data as train_dataset
train_dataset = dsets.MNIST(root = './data',
                           train = True,
                           transform = transforms.ToTensor(),
                           download = True)

#Loading the test dataset
test_dataset = dsets.MNIST(root = './data',
                           train = False,
                           transform = transforms.ToTensor())
                           

In [3]:
#Checking the length of dataset and shape of data(optional)
len(train_dataset)
train_dataset[0][0].numpy().shape

(1, 28, 28)

In [4]:
#Defining batch size and number of iterations
batch_size = 100
n_iters = 3000

In [5]:
#Finding the range for epochs
num_epochs = n_iters/(len(train_dataset) / batch_size)
num_epochs = int(num_epochs)


In [6]:
#Making the dataset iterable
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)
                                          
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)                                       
                                          
#Checking if the dataset has become iterable(optional)
import collections
isinstance(train_loader,collections.Iterable)
isinstance(test_loader,collections.Iterable)

True

In [7]:
#Creating the model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel , self).__init__()
        
        #convolution 1
        self.cnn1 = nn.Conv2d(in_channels =1, out_channels = 16 , kernel_size = 5, stride = 1,padding = 2)
        self.relu1 = nn.ReLU()
        
        #Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size = 2)
        
        #convolution 2
        self.cnn2 = nn.Conv2d(in_channels =16, out_channels = 32 , kernel_size = 5, stride = 1,padding = 2)
        self.relu2 = nn.ReLU()
        
        #Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size = 2)
        
        #fully conected layer(readout)
        self.fc = nn.Linear(32*7*7,10)
        
        
    def forward(self , x):
        #conolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        #max pool 1
        out = self.maxpool1(out)
        
        #convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        
        #max pool 2 
        out = self.maxpool2(out)
        
        #resize
        #original size :(100,32,7,7)
        #out.size(0): 100
        #New out size = (100,32*7*7)
        out = out.view(out.size(0),-1)
        
        #linear function (readout)
        out = self.fc(out)
        
        return out
 

In [8]:
model = CNNModel()

if torch.cuda.is_available():               #To put the model on GPU
    model.cuda()

In [9]:
#Defining the cross entropy loss
criterion = nn.CrossEntropyLoss()

In [10]:
#Initializing Optimizer
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters() , lr = learning_rate)

In [11]:
#Training the model
iter = 0
for epoch in range(num_epochs):
    for i ,(images ,  labels) in enumerate(train_loader):
        
        if torch.cuda.is_available():                                    #To put the variables on GPU
            
            images = Variable(images).cuda()
            labels = Variable(labels).cuda()
            
        else:
            images = Variable(images)
            labels = Variable(labels)
        
        #clearing gradient w.r.t parameters
        optimizer.zero_grad()
        
        #Forward pass to get outputs/labels
        outputs = model(images)
        
        #calulate loss using Softmax and crossentropy
        loss = criterion(outputs,labels)
        
        loss.backward()
        
        #updating parameters
        optimizer.step()
        
        iter+=1
        if iter % 500 ==0 :           #limiting the number of iterations to 500 at a time 
            total = 0
            correct = 0
            for images,labels in test_loader:
                if torch.cuda.is_available():
                    inputs = Variable(images).cuda()
                else:
                    inputs = Variable(images)
                outputs = model(inputs)
                
                _, predicted = torch.max(outputs.data,1)
                
                total += labels.size(0)
                
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
            accuracy = 100*correct/total
            #print loss
            
            print('Iteration : {},Loss : {} , Accuracy: {} '.format(iter,loss.item(),accuracy))   
                
            
        



Iteration : 500,Loss : 0.09813367575407028 , Accuracy: 96 
Iteration : 1000,Loss : 0.07711929827928543 , Accuracy: 97 
Iteration : 1500,Loss : 0.03596918657422066 , Accuracy: 97 
Iteration : 2000,Loss : 0.017032498493790627 , Accuracy: 98 
Iteration : 2500,Loss : 0.05400112271308899 , Accuracy: 98 
Iteration : 3000,Loss : 0.07246290892362595 , Accuracy: 98 
